In [ ]:
import datadoc_util as ddutil
import pims_nd2
import os
import json
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
if "DATA_DOCU_FOLDER" in env_dict:  # try default location
    data_docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    data_docu_folder = fh.open_dir("Open Data Documentation folder")

In [ ]:
ddoc = ddutil.DataDocumentation(data_docu_folder)
ddoc.loadDataDoc()

In [ ]:
if "SERVER_SYMBOL" in env_dict.keys():
    ddoc.setDataDriveSymbol(env_dict["SERVER_SYMBOL"])

In [ ]:
ddoc.GROUPING_DF.experiment_type.unique()

In [ ]:
allowed_cats = ["jrgeco_sd", "jrgeco_ctl", "tmev_bl", "tmev", "tmev_ctl", "chr2_sd", "chr2_lfpsz_sd", "chr2_szsd"]

In [ ]:
dict_nik_meta = dict()  # uuid: {"frame_rate": float, "n_frames": int, "t_begin": float, "t_end": float, time_start_jdn: float, time_start_utc: datetime}

In [ ]:
for i_row, row in ddoc.GROUPING_DF[ddoc.GROUPING_DF["experiment_type"].isin(allowed_cats)].iterrows():
    uuid = row["uuid"]
    if uuid in dict_nik_meta:
        print(f"skipping {uuid}")
        continue
    fpath = os.path.join(row["folder"], row["nd2"])
    if os.path.exists(fpath):
        try:
            nik = pims_nd2.ND2_Reader(fpath)
            fr = nik.frame_rate
            n_frames = nik.sizes["t"]
            t_begin = nik[0].metadata["t_ms"]
            t_end = nik[-1].metadata["t_ms"]
            time_start_jdn = nik.metadata["time_start_jdn"]
            time_start_utc = nik.metadata["time_start_utc"]
            nik.close()
            dict_nik_meta[uuid] = {"frame_rate": fr, "n_frames": n_frames, "t_begin": t_begin, "t_end": t_end, "time_start_jdn": time_start_jdn, "time_start_utc": time_start_utc}
        except Exception:
            print("Never mind...")

In [ ]:
for uuid in dict_nik_meta:
    dict_nik_meta[uuid]["time_start_utc"] = dict_nik_meta[uuid]["time_start_utc"].isoformat()

In [ ]:
with open(os.path.join(env_dict["DOWNLOADS_FOLDER"], "nik_metas.json"), "w") as jf:
    json.dump(dict_nik_meta, jf, indent=4)

In [ ]:
frs = [dict_nik_meta[uuid]["frame_rate"] for uuid in dict_nik_meta if dict_nik_meta[uuid]["frame_rate"] > 10]
starttimes = [datetime.fromisoformat(dict_nik_meta[uuid]["time_start_utc"]) for uuid in dict_nik_meta if  dict_nik_meta[uuid]["frame_rate"] > 10]

In [ ]:
[(uuid, dict_nik_meta[uuid]["frame_rate"]) for uuid in dict_nik_meta if dict_nik_meta[uuid]["frame_rate"] < 10]

In [ ]:
fig = plt.figure(figsize=(18, 12))
plt.scatter(starttimes, frs)
plt.ylabel("Imaging frequency (Hz)")
plt.show()